
---
# Implementation of **Value Iteration**, **Policy Iteration** and **Modified Policy Iteration(MDP)** algorithms.

The following is the MDP.py implementation from the sceleton code in A1 part 1.

---


In [66]:
import numpy as np


class MDP:
    '''A simple MDP class.  It includes the following members'''

    def __init__(self, T, R, discount):
        '''Constructor for the MDP class

        Inputs:
        T -- Transition function: |A| x |S| x |S'| array
        R -- Reward function: |A| x |S| array
        discount -- discount factor: scalar in [0,1)

        The constructor verifies that the inputs are valid and sets
        corresponding variables in a MDP object'''

        assert T.ndim == 3, "Invalid transition function: it should have 3 dimensions"
        self.nActions = T.shape[0]
        self.nStates = T.shape[1]
        assert T.shape == (
        self.nActions, self.nStates, self.nStates), "Invalid transition function: it has dimensionality " + repr(
            T.shape) + ", but it should be (nActions,nStates,nStates)"
        assert (abs(T.sum(
            2) - 1) < 1e-5).all(), "Invalid transition function: some transition probability does not equal 1"
        self.T = T
        assert R.ndim == 2, "Invalid reward function: it should have 2 dimensions"
        assert R.shape == (self.nActions, self.nStates), "Invalid reward function: it has dimensionality " + repr(
            R.shape) + ", but it should be (nActions,nStates)"
        self.R = R
        assert 0 <= discount < 1, "Invalid discount factor: it should be in [0,1)"
        self.discount = discount

    def valueIteration(self, initialV, nIterations=np.inf, tolerance=0.01):
        '''Value iteration procedure
        V <-- max_a R^a + gamma T^a V

        Inputs:
        initialV -- Initial value function: array of |S| entries
        nIterations -- limit on the # of iterations: scalar (default: infinity)
        tolerance -- threshold on ||V^n-V^n+1||_inf: scalar (default: 0.01)

        Outputs: 
        V -- Value function: array of |S| entries
        iterId -- # of iterations performed: scalar
        epsilon -- ||V^n-V^n+1||_inf: scalar'''

        V = initialV
        iterId = 0
        epsilon = np.inf
        policy = np.zeros(len(initialV), dtype=int)

        """
        print("===========================================================================")
        print("Executing Value Iteration")
        print("--------------------------------------------")
        """

        while iterId < nIterations and epsilon > tolerance:
            Ta_V = np.matmul(self.T, V)
            gamma_Ta_V = self.discount * Ta_V
            all_possible_values = self.R + gamma_Ta_V
            policy = np.argmax(all_possible_values, axis=0)  # Choose the best actions for each state, policy means keep
            V_new = np.amax((all_possible_values), axis=0)  # Choose the best action values for each state
            # np.round/np.around does not work for 0.5 so not reducing to 2 decimal places
            V_diff = (V_new - V)
            V = V_new
            epsilon = np.linalg.norm(V_diff, np.inf)
            #print("Iteration : " + str(iterId) + " V : " + str(V) + " Policy: " + str(policy) + " epsilon: " + str(epsilon))
            iterId = iterId + 1

        """
        print("--------------------------------------------")
        print("Final State values after " + str(iterId) + " iterations , V: " + str(V) + " Policy: " + str(policy))
        print("===========================================================================")
        """

        return [V, iterId, epsilon]

    def extractPolicy(self, V):
        '''Procedure to extract a policy from a value function
        pi <-- argmax_a R^a + gamma T^a V

        Inputs:
        V -- Value function: array of |S| entries

        Output:
        policy -- Policy: array of |S| entries'''

        """
        print("***************************************************************")
        print("Executing Policy Extraction")
        print("--------------------------------------------")
        """

        all_possible_values = (self.R + (self.discount*(np.matmul(self.T, V))))  # Get values for all possible state transition in this state

        """
        print("All Values for All possible state transition: ")
        print(all_possible_values)
        """

        policy = np.argmax(all_possible_values, axis=0)  # Choose the best actions for each state, policy means keep

        """
        print("Extracted Policy : " + str(policy))
        print("--------------------------------------------")
        """

        """
        # track of action chosen at timestamp t, instead of choosing only value
        max_values = [all_possible_values[policy[i]][i] for i in range(len(policy))]
        print("Values Corresponding to Selected Policies: " + str(max_values))
        print("***************************************************************")
        """

        return policy

    def evaluatePolicy(self, policy):
        '''Evaluate a policy by solving a system of linear equations
        V^pi = R^pi + gamma T^pi V^pi

        Input:
        policy -- Policy: array of |S| entries

        Ouput:
        V -- Value function: array of |S| entries'''

        """
        print("***************************************************************")
        print("Executing Policy Evaluation")
        print("--------------------------------------------")
        print("Policy : " + str(policy))
        print("Evaluating a policy by solving a system of linear equations")
        """

        R_policy = np.array([self.R[policy[i]][i] for i in range(len(policy))])
        T_policy = np.array([self.T[policy[i]][i] for i in range(len(policy))])
        gamma_T_policy = self.discount * T_policy
        assert gamma_T_policy.shape[0] == gamma_T_policy.shape[1], "gamma_T_policy matrix should be square"
        V = np.matmul(np.linalg.inv(np.identity(len(policy)) - gamma_T_policy), R_policy)

        """
        print("V : " + str(V))
        print("***************************************************************")
        """

        return V

    def policyIteration(self, initialPolicy, nIterations=np.inf):
        '''Policy iteration procedure: alternate between policy
        evaluation (solve V^pi = R^pi + gamma T^pi V^pi) and policy
        improvement (pi <-- argmax_a R^a + gamma T^a V^pi).

        Inputs:
        initialPolicy -- Initial policy: array of |S| entries
        nIterations -- limit on # of iterations: scalar (default: inf)

        Outputs: 
        policy -- Policy: array of |S| entries
        V -- Value function: array of |S| entries
        iterId -- # of iterations peformed by modified policy iteration: scalar'''

        policy = initialPolicy  # np.zeros(self.nStates)
        V = np.zeros(self.nStates)
        iterId = 0

        """
        print("===========================================================================")
        print("Executing Policy Iteration")
        print("--------------------------------------------")
        """

        while iterId < nIterations:
            V = self.evaluatePolicy(policy)
            #print("Iteration : " + str(iterId) + " V : " + str(V) + " policy : " + str(policy))

            policy_new = self.extractPolicy(V)
            iterId = iterId + 1

            if np.array_equal(policy_new, policy):
                break

            policy = policy_new

        """
        print("Iteration : " + str(iterId) + " V : " + str(V) + " policy : " + str(policy))
        print("--------------------------------------------")
        print("Final policy after " + str(iterId) + " iterations , policy: " + str(policy))
        print("===========================================================================")
        """

        return [policy, V, iterId]

    def evaluatePolicyPartially(self, policy, initialV, nIterations=np.inf, tolerance=0.01):
        '''Partial policy evaluation:
        Repeat V^pi <-- R^pi + gamma T^pi V^pi

        Inputs:
        policy -- Policy: array of |S| entries
        initialV -- Initial value function: array of |S| entries
        nIterations -- limit on the # of iterations: scalar (default: infinity)
        tolerance -- threshold on ||V^n-V^n+1||_inf: scalar (default: 0.01)

        Outputs: 
        V -- Value function: array of |S| entries
        iterId -- # of iterations performed: scalar
        epsilon -- ||V^n-V^n+1||_inf: scalar'''

        # temporary values to ensure that the code compiles until this
        # function is coded
        V = initialV  # np.zeros(self.nStates)
        iterId = 0
        epsilon = np.inf

        """
        print("***************************************************************")
        print("Executing Partial Policy Evaluation")
        print("--------------------------------------------")
        print("Policy : " + str(policy))
        print("Evaluating a policy by repeating " + str(nIterations) + " times.")
        print("Iteration : " + str(iterId) + " V : " + str(V) + " Policy: " + str(policy))
        """

        while iterId < nIterations and epsilon > tolerance:
            iterId = iterId+1
            R_policy = np.array([self.R[policy[i]][i] for i in range(len(policy))])
            T_policy = np.array([self.T[policy[i]][i] for i in range(len(policy))])
            Vnew = R_policy + (self.discount * np.matmul(T_policy, V))
            epsilon = np.linalg.norm((Vnew-V), np.inf)
            V = Vnew

            #print("Iteration : " + str(iterId) + " V : " + str(V) + " Policy: " + str(policy) + " epsilon : " + str(epsilon))

        """
        print("V : " + str(V))
        print("***************************************************************")
        """

        return [V, iterId, epsilon]

    def modifiedPolicyIteration(self, initialPolicy, initialV, nEvalIterations=5, nIterations=np.inf, tolerance=0.01):
        '''Modified policy iteration procedure: alternate between
        partial policy evaluation (repeat a few times V^pi <-- R^pi + gamma T^pi V^pi)
        and policy improvement (pi <-- argmax_a R^a + gamma T^a V^pi)

        Inputs:
        initialPolicy -- Initial policy: array of |S| entries
        initialV -- Initial value function: array of |S| entries
        nEvalIterations -- limit on # of iterations to be performed in each partial policy evaluation: scalar (default: 5)
        nIterations -- limit on # of iterations to be performed in modified policy iteration: scalar (default: inf)
        tolerance -- threshold on ||V^n-V^n+1||_inf: scalar (default: 0.01)

        Outputs: 
        policy -- Policy: array of |S| entries
        V -- Value function: array of |S| entries
        iterId -- # of iterations peformed by modified policy iteration: scalar
        epsilon -- ||V^n-V^n+1||_inf: scalar'''

        # temporary values to ensure that the code compiles until this
        # function is coded
        policy = initialPolicy
        V = initialV
        iterId = 0
        epsilon = np.inf

        """
        print("===========================================================================")
        print("Executing Modified Policy Iteration")
        print("--------------------------------------------")
        print("Iteration : " + str(iterId) + " policy : " + str(policy))
        """

        while iterId < nIterations and epsilon > tolerance:
            iterId = iterId + 1
            Vn, _ , _  = self.evaluatePolicyPartially(policy, V, nEvalIterations, tolerance)
            #print("Vn : " + str(Vn))
            all_possible_values = (self.R + (self.discount * np.matmul(self.T,Vn)))  # Get values for all possible state transition in this state
            policy = np.argmax(all_possible_values, axis=0)  # Choose the best actions for each state, policy means keep

            Vn_plus_1 = [all_possible_values[policy[i]][i] for i in range(len(policy))]
            V_diff = (Vn_plus_1 - Vn)
            V = Vn_plus_1
            epsilon = np.linalg.norm(V_diff, np.inf)
            #print("Iteration : " + str(iterId) + " policy : " + str(policy))

        """
        print("--------------------------------------------")
        print("Final policy after " + str(iterId) + " iterations , policy: " + str(policy))
        print("===========================================================================")
        """

        return [policy, V, iterId, epsilon]


## Testing value iteration, policy iteration and modified policy iteration for Markov decision processes with provided example
---

The following is the TestMDP.py file to test out the the simple MDP example from Lecture 1b Slides 17-18.

---


In [67]:
''' Construct simple MDP as described in Lecture 1b Slides 17-18'''
# Transition function: |A| x |S| x |S'| array
T = np.array([[[0.5,0.5,0,0],[0,1,0,0],[0.5,0.5,0,0],[0,1,0,0]],[[1,0,0,0],[0.5,0,0,0.5],[0.5,0,0.5,0],[0,0,0.5,0.5]]])
# Reward function: |A| x |S| array
R = np.array([[0,0,10,10],[0,0,10,10]])
# Discount factor: scalar in [0,1)
discount = 0.9        
# MDP object
mdp = MDP(T,R,discount)


---

Testing valueIteration & extractPolicy procedure using simple MDP example from Lecture 1b Slides 17-18.

---


In [68]:
'''Test each procedure'''
[V,nIterations,epsilon] = mdp.valueIteration(initialV=np.zeros(mdp.nStates))
policy = mdp.extractPolicy(V)
print("V : " + str(V))
print("nIterations : " + str(nIterations))
print("Policy : " + str(policy))

V : [31.49636306 38.51527513 43.935435   54.1128575 ]
nIterations : 58
Policy : [0 1 1 1]



---


Testing evaluatePolicy procedure using simple MDP example from Lecture 1b Slides 17-18. We see the results from evaluatePolicy match from the previous valueIteration and extractPolicy procedures

---


In [69]:
V = mdp.evaluatePolicy(np.array([1,0,1,0]))
print("Value corresponding to policy [1,0,1,0]: " + str(V))

V = mdp.evaluatePolicy(np.array([0,1,1,1]))
print("Value corresponding to policy [0,1,1,1]: " + str(V))

Value corresponding to policy [1,0,1,0]: [ 0.          0.         18.18181818 10.        ]
Value corresponding to policy [0,1,1,1]: [31.58510431 38.60401638 44.02417625 54.20159875]



---

Testing policyIteration procedure using simple MDP example from Lecture 1b Slides 17-18. We see that the policy result and value results match the valueIteration procedure.

---


In [70]:
[policy,V,iterId] = mdp.policyIteration(np.array([0,0,0,0]))
print("V : " + str(V))
print("nIterations : " + str(iterId))
print("Epsilon : " + str(policy))

V : [31.58510431 38.60401638 44.02417625 54.20159875]
nIterations : 2
Epsilon : [0 1 1 1]



---

Testing modifiedPolicyIteration(MDP) procedure using simple MDP example from Lecture 1b Slides 17-18. We see that the policy result and value results match the valueIteration procedure.

---



In [71]:
[V,iterId,epsilon] = mdp.evaluatePolicyPartially(np.array([1,0,1,0]),np.array([0,10,0,13]))
[policy,V,iterId,tolerance] = mdp.modifiedPolicyIteration(np.array([1,0,1,0]),np.array([0,10,0,13]))
print("V : " + str(V))
print("nIterations : " + str(iterId))
print("Policy : " + str(policy))

V : [31.50605365934906, 38.524965727978426, 43.945125603197766, 54.12254810271034]
nIterations : 11
Policy : [0 1 1 1]




---

## **Maze Problem** solving using value iteration, policy iteration and modified policy iteration.

---


In [72]:
import numpy as np

''' Construct a simple maze MDP

  Grid world layout:

  ---------------------
  |  0 |  1 |  2 |  3 |
  ---------------------
  |  4 |  5 |  6 |  7 |
  ---------------------
  |  8 |  9 | 10 | 11 |
  ---------------------
  | 12 | 13 | 14 | 15 |
  ---------------------

  Goal state: 14 
  Bad state: 9
  End state: 16

  The end state is an absorbing state that the agent transitions 
  to after visiting the goal state.

  There are 17 states in total (including the end state) 
  and 4 actions (up, down, left, right).'''

# Transition function: |A| x |S| x |S'| array
T = np.zeros([4, 17, 17])
a = 0.7;  # intended move
b = 0.15;  # lateral move

# up (a = 0)

T[0, 0, 0] = a + b;
T[0, 0, 1] = b;

T[0, 1, 0] = b;
T[0, 1, 1] = a;
T[0, 1, 2] = b;

T[0, 2, 1] = b;
T[0, 2, 2] = a;
T[0, 2, 3] = b;

T[0, 3, 2] = b;
T[0, 3, 3] = a + b;

T[0, 4, 4] = b;
T[0, 4, 0] = a;
T[0, 4, 5] = b;

T[0, 5, 4] = b;
T[0, 5, 1] = a;
T[0, 5, 6] = b;

T[0, 6, 5] = b;
T[0, 6, 2] = a;
T[0, 6, 7] = b;

T[0, 7, 6] = b;
T[0, 7, 3] = a;
T[0, 7, 7] = b;

T[0, 8, 8] = b;
T[0, 8, 4] = a;
T[0, 8, 9] = b;

T[0, 9, 8] = b;
T[0, 9, 5] = a;
T[0, 9, 10] = b;

T[0, 10, 9] = b;
T[0, 10, 6] = a;
T[0, 10, 11] = b;

T[0, 11, 10] = b;
T[0, 11, 7] = a;
T[0, 11, 11] = b;

T[0, 12, 12] = b;
T[0, 12, 8] = a;
T[0, 12, 13] = b;

T[0, 13, 12] = b;
T[0, 13, 9] = a;
T[0, 13, 14] = b;

T[0, 14, 16] = 1;

T[0, 15, 11] = a;
T[0, 15, 14] = b;
T[0, 15, 15] = b;

T[0, 16, 16] = 1;

# down (a = 1)

T[1, 0, 0] = b;
T[1, 0, 4] = a;
T[1, 0, 1] = b;

T[1, 1, 0] = b;
T[1, 1, 5] = a;
T[1, 1, 2] = b;

T[1, 2, 1] = b;
T[1, 2, 6] = a;
T[1, 2, 3] = b;

T[1, 3, 2] = b;
T[1, 3, 7] = a;
T[1, 3, 3] = b;

T[1, 4, 4] = b;
T[1, 4, 8] = a;
T[1, 4, 5] = b;

T[1, 5, 4] = b;
T[1, 5, 9] = a;
T[1, 5, 6] = b;

T[1, 6, 5] = b;
T[1, 6, 10] = a;
T[1, 6, 7] = b;

T[1, 7, 6] = b;
T[1, 7, 11] = a;
T[1, 7, 7] = b;

T[1, 8, 8] = b;
T[1, 8, 12] = a;
T[1, 8, 9] = b;

T[1, 9, 8] = b;
T[1, 9, 13] = a;
T[1, 9, 10] = b;

T[1, 10, 9] = b;
T[1, 10, 14] = a;
T[1, 10, 11] = b;

T[1, 11, 10] = b;
T[1, 11, 15] = a;
T[1, 11, 11] = b;

T[1, 12, 12] = a + b;
T[1, 12, 13] = b;

T[1, 13, 12] = b;
T[1, 13, 13] = a;
T[1, 13, 14] = b;

T[1, 14, 16] = 1;

T[1, 15, 14] = b;
T[1, 15, 15] = a + b;

T[1, 16, 16] = 1;

# left (a = 2)

T[2, 0, 0] = a + b;
T[2, 0, 4] = b;

T[2, 1, 1] = b;
T[2, 1, 0] = a;
T[2, 1, 5] = b;

T[2, 2, 2] = b;
T[2, 2, 1] = a;
T[2, 2, 6] = b;

T[2, 3, 3] = b;
T[2, 3, 2] = a;
T[2, 3, 7] = b;

T[2, 4, 0] = b;
T[2, 4, 4] = a;
T[2, 4, 8] = b;

T[2, 5, 1] = b;
T[2, 5, 4] = a;
T[2, 5, 9] = b;

T[2, 6, 2] = b;
T[2, 6, 5] = a;
T[2, 6, 10] = b;

T[2, 7, 3] = b;
T[2, 7, 6] = a;
T[2, 7, 11] = b;

T[2, 8, 4] = b;
T[2, 8, 8] = a;
T[2, 8, 12] = b;

T[2, 9, 5] = b;
T[2, 9, 8] = a;
T[2, 9, 13] = b;

T[2, 10, 6] = b;
T[2, 10, 9] = a;
T[2, 10, 14] = b;

T[2, 11, 7] = b;
T[2, 11, 10] = a;
T[2, 11, 15] = b;

T[2, 12, 8] = b;
T[2, 12, 12] = a + b;

T[2, 13, 9] = b;
T[2, 13, 12] = a;
T[2, 13, 13] = b;

T[2, 14, 16] = 1;

T[2, 15, 11] = b;
T[2, 15, 14] = a;
T[2, 15, 15] = b;

T[2, 16, 16] = 1;

# right (a = 3)

T[3, 0, 0] = b;
T[3, 0, 1] = a;
T[3, 0, 4] = b;

T[3, 1, 1] = b;
T[3, 1, 2] = a;
T[3, 1, 5] = b;

T[3, 2, 2] = b;
T[3, 2, 3] = a;
T[3, 2, 6] = b;

T[3, 3, 3] = a + b;
T[3, 3, 7] = b;

T[3, 4, 0] = b;
T[3, 4, 5] = a;
T[3, 4, 8] = b;

T[3, 5, 1] = b;
T[3, 5, 6] = a;
T[3, 5, 9] = b;

T[3, 6, 2] = b;
T[3, 6, 7] = a;
T[3, 6, 10] = b;

T[3, 7, 3] = b;
T[3, 7, 7] = a;
T[3, 7, 11] = b;

T[3, 8, 4] = b;
T[3, 8, 9] = a;
T[3, 8, 12] = b;

T[3, 9, 5] = b;
T[3, 9, 10] = a;
T[3, 9, 13] = b;

T[3, 10, 6] = b;
T[3, 10, 11] = a;
T[3, 10, 14] = b;

T[3, 11, 7] = b;
T[3, 11, 11] = a;
T[3, 11, 15] = b;

T[3, 12, 8] = b;
T[3, 12, 13] = a;
T[3, 12, 12] = b;

T[3, 13, 9] = b;
T[3, 13, 14] = a;
T[3, 13, 13] = b;

T[3, 14, 16] = 1;

T[3, 15, 11] = b;
T[3, 15, 15] = a + b;

T[3, 16, 16] = 1;

# Reward function: |A| x |S| array
R = -1 * np.ones([4, 17]);

# set rewards
R[:, 14] = 100;  # goal state
R[:, 9] = -70;  # bad state
R[:, 16] = 0;  # end state

# Discount factor: scalar in [0,1)
discount = 0.95

# MDP object
mdp = MDP(T, R, discount)




---

### Calculating **policy**, **value function** and **number of iterations** needed by **value iteration** in Maze Problem

---
#### Testing Parameters

*   Tolerance : 0.01. Used following line in *mdp.valueIteration* procedure
```
#  tolerance=0.01
```
*   Initial Value Function: Zero for all states. Used following line in *mdp.valueIteration* procedure
```
#  initialV=np.zeros(mdp.nStates)
```

---

#### Results

1.   **Policy** : [3 3 1 1 3 0 1 1 1 3 3 1 3 3 0 2 0]
2.   **Value Function** : [ 49.66895295  55.27161487  61.57363146  65.87460207  48.00953133 52.30188839  68.13923599  73.25481858  50.22617115  -0.42481753
  77.06611566  81.36344171  66.36179067  76.31383816 100.
  89.90583635   0.        ]
3.   **Number of Iterations** : 26

---


In [73]:
[V, nIterations, epsilon] = mdp.valueIteration(initialV=np.zeros(mdp.nStates), tolerance=0.01)
policy = mdp.extractPolicy(V)

In [74]:
print("==================================================================")
print("MDPMaze Value Iteration:")
print("Policy " + str(policy))
print("Value Function: " + str(V))
print("Number of Iterations: " + str(nIterations))
print("==================================================================\n\n")


MDPMaze Value Iteration:
Policy [3 3 1 1 3 0 1 1 1 3 3 1 3 3 0 2 0]
Value Function: [ 49.66895295  55.27161487  61.57363146  65.87460207  48.00953133
  52.30188839  68.13923599  73.25481858  50.22617115  -0.42481753
  77.06611566  81.36344171  66.36179067  76.31383816 100.
  89.90583635   0.        ]
Number of Iterations: 26





---

### Calculating **policy**, **value function** and **number of iterations** needed by **policy iteration** in Maze Problem
---
#### Testing Parameters

*   Initial Policy: Uses action 0 in all states. Used following line in *mdp.policyIteration* procedure
```
#  initialPolicy = np.zeros(mdp.nStates, dtype=int)
```

---

#### Results


1.   **Policy** : [3 3 1 1 3 0 1 1 1 3 3 1 3 3 0 2 0]
2.   **Value Function** : [ 49.69078867  55.28617892  61.58230087  65.87897994  48.03187576  52.32047965  68.1447605   73.25676304  50.23031164  -0.41942079
  77.06767431  81.36397885  66.36430029  76.31513999 100.
  89.90596733   0.        ]
3.   **Number of Iterations** : 7

---



In [75]:
[policy, V, nIterations] = mdp.policyIteration(initialPolicy = np.zeros(mdp.nStates, dtype=int))


In [76]:
print("==================================================================")
print("MDPMaze policy Iteration:")
print("Policy " + str(policy))
print("Value Function: " + str(V))
print("Number of Iterations: " + str(nIterations))
print("==================================================================\n\n")

MDPMaze policy Iteration:
Policy [3 3 1 1 3 0 1 1 1 3 3 1 3 3 0 2 0]
Value Function: [ 49.69078867  55.28617892  61.58230087  65.87897994  48.03187576
  52.32047965  68.1447605   73.25676304  50.23031164  -0.41942079
  77.06767431  81.36397885  66.36430029  76.31513999 100.
  89.90596733   0.        ]
Number of Iterations: 7





---

### Calculating **number of iterations** to converge when varying number of iterations in partial policy evaluation from 1 to 10 in Maze Problem

---

#### Testing Parameters

*   Tolerance : 0.01 . Used following line in *mdp.modifiedPolicyIteration* procedure
```
#  tolerance=0.01
```
*   Initial Policy : Uses action 0 in all states . Used following line in *mdp.modifiedPolicyIteration* procedure
```
#  initialPolicy = np.zeros(mdp.nStates, dtype=int)
```
*   Initial Value Function : Assigned 0 to in all states . Used following line in *mdp.modifiedPolicyIteration* procedure
```
#  initialPolicy = initialV = np.zeros(mdp.nStates)
```
*   Number of iterations in Partial Policy Evaluation : 0 to 10
---

#### Results

| Number of Iterations in Partial Policy Evaluation | Number of Iterations for Convergence |
| --- | --- | 
| 1 | 14 | 
| 2 | 10 | 
| 3 | 8 | 
| 4 | 8 | 
| 5 | 7 | 
| 6 | 7 | 
| 7 | 7 | 
| 8 | 6 | 
| 9 | 7 | 
| 10 | 7 | 


---


In [79]:

print("==================================================================")
print("MDPMaze Modifier Policy Iteration:")
for nIterPPE in range(1, 11):
    [policy, V, nIterations, epsilon] = mdp.modifiedPolicyIteration(initialPolicy = np.zeros(mdp.nStates, dtype=int),
                                                                    initialV = np.zeros(mdp.nStates), nEvalIterations=nIterPPE,
                                                                    tolerance=0.01)
    print("# of Iterations in Partial Policy Evaluation: " + str(nIterPPE) + " , # of Iterations for Convergence: " + str(nIterations))
print("==================================================================\n\n")


MDPMaze Modifier Policy Iteration:
# of Iterations in Partial Policy Evaluation: 1 , # of Iterations for Convergence: 14
# of Iterations in Partial Policy Evaluation: 2 , # of Iterations for Convergence: 10
# of Iterations in Partial Policy Evaluation: 3 , # of Iterations for Convergence: 8
# of Iterations in Partial Policy Evaluation: 4 , # of Iterations for Convergence: 8
# of Iterations in Partial Policy Evaluation: 5 , # of Iterations for Convergence: 7
# of Iterations in Partial Policy Evaluation: 6 , # of Iterations for Convergence: 7
# of Iterations in Partial Policy Evaluation: 7 , # of Iterations for Convergence: 7
# of Iterations in Partial Policy Evaluation: 8 , # of Iterations for Convergence: 6
# of Iterations in Partial Policy Evaluation: 9 , # of Iterations for Convergence: 7
# of Iterations in Partial Policy Evaluation: 10 , # of Iterations for Convergence: 7




---
##Discussion
---
| Convergence Iterations in Value Iteration | Convergence Iterations in Policy Iteration | Convergence Iterations in Partial Policy Iteration | 
| --- | --- | --- | 
| 26 | 7 | 6 (For 8 iterations in partial policy evaluation) | 

In the maze problem, there are a total of |S| = 17 states and |A| = 4 actions. For MDP problem, we see that a k=8 value produces the least number of iterations (6) for convergence for given range of k values (1 to 10). **To recap, k denotes the number of iterations in partial policy evaluation step for MDP.**

Increases the number of iterations in partial policy evaluation results in slowly decreasing the number of iterations in MDP for convergence.

As number of Actions |A| (= 4) < number of states |S| (=17), runtime for each iteration in 
*   **Value Iteration** reduces to O(|S|<sup>2</sup>|A|)
*   **Policy Iteration** reduces to O(|S|<sup>3</sup>) 
[As |S| > |A|, |S|<sup>3</sup> > |S|<sup>2</sup>|A| and O(|S|<sup>3</sup> + |S|<sup>2</sup>|A|) reduces to O(|S|<sup>3</sup>)]  
*   **MDP** reduces to O(k|S|<sup>2</sup>) when k > |A| (number of partial policy evaluation >= 5) and  O(|S|<sup>2</sup>|A|) when k <= |A| (number of partial policy evaluation is between 1 to 4 inclusive)

For **k > 4**, each iteration in MDP will be less complex than policy iteration (k|S|<sup>2</sup> < |S|<sup>2</sup>) until k > |S|, or number of iterations in partial policy evaluation crosses 17. However, between 4 < k < 17, each iteration in MDP will still be more complex than value iteration. 

For **k <= 4**, each iteration in MDP will be less complex than policy iteration [O(k|S|<sup>2</sup>) < O(|S|<sup>2</sup>)] and value iteration [O(k|S|<sup>2</sup>) < O(|S|<sup>2</sup>|A|)]. 
